# Tests for Linearity Tab

In [2]:
import pydicom
from pylinac import FieldAnalysis

In [3]:
file = "RI_test_linearity.dcm"
ds = pydicom.dcmread(file)

In [4]:
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 182
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: RT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.246.352.81.3.163227933.43057.19624.180.101
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.246.352.70.2.1.160.3
(0002, 0013) Implementation Version Name         SH: 'DCIE 16.1'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'PORTAL', 'ACQUIRED_DOSE']
(0008, 0012) Instance Creation Date              DA: '20230731'
(0008, 0013) Instance Creation Time              TM: '124317'
(0008, 0016) SOP Class UID                       UI: RT Image Storage
(0008, 0018) SOP Ins

In [5]:
int(ds.ExposureSequence[0].MetersetExposure)

100

In [6]:
my_img = FieldAnalysis(file)
my_img.analyze(vert_width = 0.1, horiz_width = 0.1)

print(my_img.results())

Field Analysis Results
----------------------
File: RI_test_linearity.dcm
Protocol: VARIAN
Centering method: Beam center
Normalization method: Beam center
Interpolation: Linear
Edge detection method: Inflection Derivative

Penumbra width (20/80):
Left: 2.1mm
Right: 2.1mm
Top: 2.9mm
Bottom: 2.8mm

Field Size:
Horizontal: 99.8mm
Vertical: 99.4mm

CAX to edge distances:
CAX -> Top edge: 47.9mm
CAX -> Bottom edge: 51.5mm
CAX -> Left edge: 50.1mm
CAX -> Right edge: 49.8mm

Central ROI stats:
Mean: 101.59309475140992
Max: 103.13362810800001
Min: 99.845445042
Standard deviation: 0.6001688309380391

Top slope: 0.056%/mm
Bottom slope: 0.007%/mm
Left slope: 0.010%/mm
Right slope: -0.031%/mm

Protocol data:
--------------
Vertical symmetry: -3.028%
Horizontal symmetry: 0.831%

Vertical flatness: 2.141%
Horizontal flatness: 1.136%



## Test for uniformity

In [8]:
from pylinac import FieldAnalysis
file = "RI_test_uniformity.dcm"

my_img = FieldAnalysis(file)
my_img.analyze(vert_width = 0.1, horiz_width = 0.1)
print(my_img.results())

C:\Users\alfon\miniconda3\envs\pyside6\lib\site-packages\pylinac\core\profile.py:1634: PeakPropertyWarning: some peaks have a width of 0
  peak_idxs, peak_props = signal.find_peaks(


ValueError: Inputs must not be empty.

In [61]:
from pylinac.core import image

from skimage.draw import rectangle
from skimage import measure

class UniformityAnalysis:
    def __init__(self, path):
        self._path: str = path
        self.image: image.ImageLike = image.load(path)
        self._is_analyzed = False

    def _get_vert_idx(
        self, vert_position: float, vert_width: float
        ) -> (float, float):
        left_edge = int(
        round(
            self.image.array.shape[1] * vert_position
            - self.image.array.shape[1] * vert_width / 2
            )
        )
        left_edge = max(left_edge, 0)  # clip to 0
        right_edge = int(
            round(
                self.image.array.shape[1] * vert_position
                + self.image.array.shape[1] * vert_width / 2
            )
            + 1
        )
        right_edge = min(right_edge, self.image.array.shape[1])  # clip to image limit
        return (
            left_edge,
            right_edge,
        )

    def _get_horiz_idx(
        self, horiz_position: float, horiz_width: float
        ) -> (float, float):
        
        bottom_edge = int(
            round(
                self.image.array.shape[0] * horiz_position
                - self.image.array.shape[0] * horiz_width / 2
            )
        )
        bottom_edge = max(bottom_edge, 0)
        top_edge = int(
            round(
                self.image.array.shape[0] * horiz_position
                + self.image.array.shape[0] * horiz_width / 2
            )
            + 1
        )
        top_edge = min(top_edge, self.image.array.shape[0])
        return (
            bottom_edge,
            top_edge,
        )
    
    def get_uniformity(
        self, 
        horiz_position = 0.5,
        horiz_width = 0.85,
        vert_position = 0.5,
        vert_width = 0.85):
        """Calculate the central ROI uniformity."""
        
        left_v_idx, right_v_idx = self._get_vert_idx(vert_position, vert_width)
        upper_h_idx, lower_h_idx = self._get_horiz_idx(horiz_position, horiz_width)
        
        width =  max(abs(left_v_idx - right_v_idx), 2)
        height = max(abs(upper_h_idx - lower_h_idx), 2)

        label_rectangle = np.zeros(self.image.shape, dtype = np.uint8)
        start = (left_v_idx, upper_h_idx)
        end = (left_v_idx + height, upper_h_idx + width)
        rr, cc = rectangle(start, end = end, shape=label_rectangle.shape)
        label_rectangle[rr, cc] = 1

        def stdev(region, intensities):
            # note the ddof arg to get the sample var if you so desire!
            return np.std(intensities[region], ddof=1)

        props = measure.regionprops(label_rectangle, intensity_image=self.image.array, extra_properties=[stdev])
        print(props[0].intensity_mean)
                
        self.mean = props[0].intensity_mean
        self.std = props[0].stdev
        self.num_pixels = props[0].num_pixels

        self.uniformity = self.std / self.mean * 100
        self._is_analyzed = True
        return self.uniformity

    def results(self, as_str=True) -> str:
        """Get the results of the analysis.

        Parameters
        ----------
        as_str
            If True, return a simple string. If False, return a list of each line of text.
        """
        if not self._is_analyzed:
            raise NotAnalyzed("Image is not analyzed yet. Use get_uniformity() first.")

        results = [
            "Uniformity Analysis Results",
            "---------------------------",
            f"File: {self._path}",
        ]
        results += [
                "Central ROI stats:",
                f"Mean: {self.mean}",
                f"Standard deviation: {self.std}",
                f"Uniformity [%]: {self.uniformity}",
                f"Num. pixels: {self.num_pixels}",
            ]
        return results


In [62]:
file = "RI_test_uniformity.dcm"

my_img = UniformityAnalysis(file)
uniformity = my_img.get_uniformity(vert_width = 0.85, horiz_width = 0.85)
my_img.results()
#print(my_img.uniformity)

119.44502008676693


['Uniformity Analysis Results',
 '---------------------------',
 'File: RI_test_uniformity.dcm',
 'Central ROI stats:',
 'Mean: 119.44502008676693',
 'Standard deviation: 1.7634332388628446',
 'Uniformity [%]: 1.4763555965597026',
 'Num. pixels: 570288']

In [63]:
print(uniformity)
print(props[0].num_pixels)

1.4763555965597026
9


In [1]:
import numpy as np
from skimage.draw import rectangle

In [15]:
label = np.zeros((5,5), dtype = np.uint8)
start = (1,1)
end = (3,3)
rr, cc = rectangle(start, end = end, shape=label.shape)
label[rr, cc] = 1
label

array([[0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]], dtype=uint8)

In [28]:
from skimage import measure
img = label * 2
print(img)

def image_stdev(region, intensities):
    # note the ddof arg to get the sample var if you so desire!
    return np.std(intensities[region], ddof=1)

props = measure.regionprops(label, intensity_image=img, extra_properties=[image_stdev])
props[0].intensity_mean

[[0 0 0 0 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 0 0 0 0]]


2.0

In [29]:
props[0].image_stdev

0.0

In [30]:
props[0].num_pixels

9

In [31]:
props[0].centroid

(2.0, 2.0)